In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from quantopian.pipeline.filters import Q500US
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# Pairs Trading Strategy
## Shuyu Hao, Yumeng Zhang, Zheng Li, Xiangyuan Xie
### 2020-6-22

#### General Defination: 
Pairs trading is to find stock pairs that are highly correlated and to trade the stock price spreads: short the spread when the spread is above a certain bound; long the spread when the spread is below a certain bound.
#### Basic Assumption: 
Highly correlated stock pairs' price spreads will fluctuate in a certain range with a certain mean.
#### Research Process:
Find and test those stock pairs which accord with our general defination and have high cointegration.

## Step 1 Building Data Set

#### Selecting data
Choose the ticker symbols of top 100 Q500US stocks which have the largest dollar volume and make pipeline

In [ ]:
dollar_volume = AverageDollarVolume(window_length =1)
high_dollar_volume = dollar_volume.top(100, mask = Q500US())
my_symbol = Fundamentals.symbol.latest 
sector = Fundamentals.morningstar_sector_code.latest
my_pipe = Pipeline(
        columns={'my_symbol' : my_symbol, 
                 'sector': sector},
        screen = high_dollar_volume)

In [ ]:
results = run_pipeline(my_pipe, '2014-01-01', '2014-01-01')
results = results.xs('2014-01-02')
symbol_list = results.my_symbol.tolist()
symbol_list.remove('GOOGL')
results

## Step 2 Get cointegrated pairs

#### Cointegration simply tests if two stocks' price spreads fluctuate with a certain mean

In [ ]:
# Define function to get conintegrated pairs
def find_cointegrated_pairs(data, df):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    correlation_matrix = np.ones((n,n))
    keys = data.keys()
    pairs = []
    pairs1 = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            sec1 = df['sector'][i]
            sec2 = df['sector'][j]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.01 and sec1 == sec2: 
                pairs.append((keys[i]))
                pairs.append(keys[j])
                pairs1.append((keys[i], keys[j]))
                print('(' + 'symbol' + '(' + "'"+ keys[i] +"'" + ')' + ',' + 'symbol' + '(' + "'" + keys[j] +"'"+ ')'+ ')' + ',')
    return score_matrix, pvalue_matrix, pairs, pairs1

In [ ]:
# Get pricing dataframe
prices = get_pricing(symbol_list, fields=['price']
                               , start_date='2014-01-01', end_date='2015-01-01')['price']
prices.columns = map(lambda x: x.symbol, prices.columns)
prices

In [ ]:
# print results
scores, pvalues, pairs, pairs1 = find_cointegrated_pairs(prices, results)
print pairs
print('we found ' + str(len(pairs1))+ ' pairs in the dataset')

## Step 3 Visualize Cointegration 

In [ ]:
#Pvalues
for i in pairs1:
    S1 = prices[i[0]]
    S2 = prices[i[1]]
    score, pvalue, _ = coint(S1, S2)
    print pvalue

#### Example

In [ ]:
S1 = prices['DAL']
S2 = prices['MMM']

#### Regression 
Regression tells us how the price change of one stock is related to that of another stock. Theoratically, we want to hedge the beta between two stocks and only concentrate on the pure alpha to reduce dollar exposure, but sometimes dollar exposure can bring profits.

In [ ]:
# Regress S2 on S1 and get the beta, which is the hedge ratio
S1 = sm.add_constant(S1)
results = sm.OLS(S2, S1).fit()
S1 = S1['DAL']
b = results.params['DAL']
S2.plot(legend = True)
S1.plot(legend = True)

In [ ]:
#Plot the price pure spread to see if it behaves like a random noise
spread = S2 - b * S1
spread.plot()
plt.axhline(spread.mean(), color='black')
plt.legend(['Spread'])

#### Z-score
Usually we want to standardize a series to find specific bound or signals for us to trade

In [ ]:
#Ploting Z-score
def zscore(series):
    return (series - series.mean()) / np.std(series)

In [ ]:
zscore(spread).plot()
plt.axhline(zscore(spread).mean(), color='black')
plt.axhline(1.0, color='red', linestyle='--')
plt.axhline(-1.0, color='green', linestyle='--')
plt.legend(['Spread z-score', 'Mean', '+1', '-1']);

## Step 4 Using the most recent data 
Even the spread we calculated behaves like a random alpha, the volatility of the spread will change. Therefore,we roll the most recent data to:

### Run regression
get rolling beta

In [ ]:
#Calculating Rolling Beta
rolling_beta = pd.ols(y=S2, x=S1, window_type='rolling', window=30)
spread = S2 - rolling_beta.beta['x'] * S1
spread.name = 'spread'

### Calculate Z-scores 
get rolling z-scores

In [ ]:
# Get the 1 day moving average of the price spread
mavg1 = pd.rolling_mean(spread, window=1)
mavg1.name = 'spread 1d mavg'

# Get the 30 day moving average of the price spread
mavg30 = pd.rolling_mean(spread, window=30)
mavg30.name = 'spread 30d mavg'

std30 = pd.rolling_std(spread, window=30)
std30.name = 'std 30d'
#Calculate rolling zscore
z_30_1 = (mavg1 - mavg30)/std30
z_30_1.name = 'z-score'

In [ ]:
plt.plot(mavg1.index, mavg1.values)
plt.plot(mavg30.index, mavg30.values)

plt.legend(['1-Day Spread MAVG', '30-Day Spread MAVG'])
plt.ylabel('Spread');

##### for this pair of stocks, a bound of 1 may not be the best choice as the trading signal, 2 would be better

In [ ]:
z_30_1.plot()
plt.axhline(0, color='black')
plt.axhline(1.0, color='red', linestyle='--');
plt.axhline(2.0, color='green', linestyle='--')

### Algorithm

In [ ]:
import numpy as np
import pandas as pd
import quantopian.optimize as opt
import quantopian.algorithm as algo
import statsmodels.api as sm

 
MAX_GROSS_EXPOSURE = 1.0 # Set exposure constraint constant value for optimizer
    
def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    schedule_function(check_pair_status, date_rules.every_day(), time_rules.market_close(minutes=30))
    
  

    context.stocks = [(symbol('MMM'),symbol('DAL')),
                      (symbol('DDD'),symbol('P')),
                      (symbol('UPS'),symbol('DAL'))]

    # Our threshold for trading on the z-score
    context.entry_threshold = 2
    context.exit_threshold = 0.05
    
    # Create a variable to store our target weights
    context.target_weights = pd.Series(index=context.stocks, data=0.0)
    
    # Moving average lengths
    context.long_ma_length = 30
    context.short_ma_length = 1
    
    # Flags to tell us if we're currently in a trade
    context.currently_long_the_spread = False
    context.currently_short_the_spread = False


def check_pair_status(context, data):
    
    for pair in context.stocks:
    # For notational convenience
        s1 = pair[0]
        s2 = pair[1]
    # Get pricing history
        prices = data.history([s1, s2], "price", context.long_ma_length, '1d')
        
        #try:
            #hedge_r = hedge_ratio(prices[s1], prices[s2], add_const=True)      
        #except ValueError as e:
            #log.debug(e)
            #return
    

        short_prices = prices.iloc[-context.short_ma_length:]
    
    # Get the long mavg
        long_ma = np.mean(prices[s1] - prices[s2])
                          #hedge_r * prices[s2])
    # Get the std of the long window
        long_std = np.std(prices[s1] - prices[s2])
                          #hedge_r * prices[s2])
        
    
    
    # Get the short mavg
        short_ma = np.mean(short_prices[s1] - short_prices[s2])
                           #hedge_r * short_prices[s2])
    
    # Compute z-score
        if long_std > 0:
            zscore = (short_ma - long_ma)/long_std
    
        # Our two entry cases
            if zscore > context.entry_threshold and \
                not context.currently_short_the_spread:
                context.target_weights[s1] = -1/(2*len(context.stocks)) # short top
                context.target_weights[s2] = 1/(2*len(context.stocks))#*hedge_r # long bottom
                context.currently_short_the_spread = True
                context.currently_long_the_spread = False
                allocate(context, data)
                return
            
            elif zscore < -context.entry_threshold and \
                not context.currently_long_the_spread:
                context.target_weights[s1] = 1/(2*len(context.stocks)) 
                context.target_weights[s2] = -1/(2*len(context.stocks))#*hedge_r
                context.currently_short_the_spread = False
                context.currently_long_the_spread = True
                allocate(context, data)
                return
            
        # Our exit case
            elif abs(zscore) < context.exit_threshold:
                context.target_weights[s1] = 0 # close out
                context.target_weights[s2] = 0 # close out
                context.currently_short_the_spread = False
                context.currently_long_the_spread = False
                allocate(context, data)
                return
                
            record('zscore', zscore)
    
    # Call the tra
    
#def hedge_ratio(Y, X, add_const=True):
    #if add_const:
        #X = sm.add_constant(X)
        #model = sm.OLS(Y, X).fit()
        #return model.params[1]
    #model = sm.OLS(Y, X).fit()
    #return model.params.values        
        
def allocate(context, data):    
    # Set objective to match target weights as closely as possible, given constraints
    objective = opt.TargetWeights(context.target_weights)
    
    # Define constraints
    constraints = []
    constraints.append(opt.MaxGrossExposure(MAX_GROSS_EXPOSURE))
    
    algo.order_optimal_portfolio(
        objective=objective,
        constraints=constraints,
    )